In [3]:
import numpy as np
from typing import Tuple

In [ ]:
import tensorflow as tf

def create_lstm_model(x:int, input_shape:Tuple[int, int]) -> tf.keras.Model:
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(x, input_shape=(), activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LSTM(128, input_shape=(), activation="relu"),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.LSTM(56, input_shape=(), activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(), 
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(56, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        loss="mae",
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.003, 
                                           beta_1=0.9, 
                                           beta_2=0.999, 
                                           epsilon=1e-7),
        metrics=["acc", "val_acc"]
    )

    return model